_VALIDADOR DE ARCHIVOS PLANOS Y DETECCIÓN DE ANOMALÍAS_


**ACTUALIZAR EL TIPO DE CODIFICACIÓN**


In [2]:
# Importar módulos
import os                                                                                           # Módulo para interactuar con el sistema operativo
import re                                                                                            # Módulo para trabajar con expresiones regulares   
import chardet                                                                                 # Módulo para detectar la codificación de un archivo
import locale                                                                                    # Módulo para trabajar con la configuración regional del sistema                                 
import numpy as np                                                                        # Módulo para trabajar con arreglos y matrices                 
import pandas as pd                                                                       # Módulo para trabajar con estructuras de datos                      
import matplotlib.pyplot as plt                                                    # Módulo para realizar gráficos                 
import seaborn as sns                                                                    # Módulo para realizar gráficos                                   
from sklearn.ensemble import IsolationForest                         # Módulo para trabajar con el algoritmo Isolation Forest                  
from sklearn.preprocessing import StandardScaler                # Módulo para estandarizar los datos                    
from sklearn.svm import OneClassSVM                                      # Módulo para trabajar con el algoritmo One-Class SVM                   
#from tensorflow.keras.models import Sequential                      # Módulo para trabajar con modelos secuenciales de Keras                      Pendiente de instalar por cambio de versión de Python
#from tensorflow.keras.layers import Dense                                 # Módulo para trabajar con capas densas de Keras                                     Pendiente de instalar por cambio de versión de Python
from sklearn.neighbors import LocalOutlierFactor                   # Módulo para trabajar con el algoritmo Local Outlier Factor                              
from sklearn.covariance import EllipticEnvelope                      # Módulo para trabajar con el algoritmo Elliptic Envelope                       
from sklearn.cluster import KMeans                                            # Módulo para trabajar con el algoritmo K-Means                                   
import warnings                                                                                # Módulo para suprimir advertencias                             

# Suprimir FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Carpeta donde se encuentran los archivos a procesar
carpeta = (r"D:\Usuarios\14624165\Desktop\Planos nomina\12 diciembre")

def convertir_a_cp1252(ruta_archivo):
    codificaciones = ['utf-8', 'latin-1', 'iso-8859-1']
    contenido = None

    for codificacion in codificaciones:
        try:
            with open(ruta_archivo, 'r', encoding=codificacion) as archivo:
                contenido = archivo.read()
            break
        except UnicodeDecodeError:
            continue

    if contenido is not None:
        with open(ruta_archivo, 'w', encoding='cp1252', errors='replace') as archivo:
            archivo.write(contenido)

def procesar_carpetas(carpeta):
    for carpeta_raiz, _, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith('.txt'):
                ruta_archivo = os.path.join(carpeta_raiz, archivo)
                convertir_a_cp1252(ruta_archivo)

procesar_carpetas(carpeta)

**VALIDADOR DE PLANO**


In [ ]:
# Configurar la localización para formatear los números como moneda
locale.setlocale(locale.LC_ALL, '')

# Variable globales
F350_ID_TIPO_DOCTO_GLOBAL = None
suma_valor_db_tipo4 = 0.0
suma_valor_cr_tipo4 = 0.0
consecutivo_esperado = 1
valores_DB_CR = {
    'F351_VALOR_DB': [],
    'F351_VALOR_DB2': [],
    'F351_VALOR_DB3': [],
    'F351_VALOR_CR': [],
    'F351_VALOR_CR2': [],
    'F351_VALOR_CR3': []
}

# Valores de terceros
valores_validos_tercero_ps = [
    "890904996", "800112806", "830113831", "800130907", "800140949",
    "800251440", "860066942", "800088702", "890303093", "800250119",
    "805000427", "830003564", "805001157", "830009783", "830074184",
    "900156264", "900074992", "900462447", "890102257", "890203183",
    "890399010", "899999063", "891500319", "890480123", "890980040",
    "891080031", "800118954", "891800330", "800229739", "800224808",
    "800253055", "830125132", "800227940", "900336004", "800256161",
    "890303208", "899999054", "899999239", "899999001", "899999034",
    "890903790", "901037916", "900226715", "900935126", "837000084",
    "901021565"
]

# Función para validar que F_NUMERO_REG sea consecutivo
def validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG):
    global consecutivo_esperado
    errores = []
    
    if int(F_NUMERO_REG) != consecutivo_esperado:
        errores.append(f"F_NUMERO_REG {F_NUMERO_REG} no es consecutivo. Se esperaba {consecutivo_esperado}.")
    
    consecutivo_esperado += 1
    return errores

# Validación para que F351_VALOR_DB, F351_VALOR_DB2 y F351_VALOR_DB3 sean iguales
def validar_igualdad_valores_DB(F351_VALOR_DB, F351_VALOR_DB2, F351_VALOR_DB3):
    errores = []
    if F351_VALOR_DB != F351_VALOR_DB2 or F351_VALOR_DB != F351_VALOR_DB3:
        errores.append("F351_VALOR_DB, F351_VALOR_DB2 y F351_VALOR_DB3 no son iguales.")
    return errores

# Validación para que F351_VALOR_CR, F351_VALOR_CR2 y F351_VALOR_CR3 sean iguales
def validar_igualdad_valores_CR(F351_VALOR_CR, F351_VALOR_CR2, F351_VALOR_CR3):
    errores = []
    if F351_VALOR_CR != F351_VALOR_CR2 or F351_VALOR_CR != F351_VALOR_CR3:
        errores.append("F351_VALOR_CR, F351_VALOR_CR2 y F351_VALOR_CR3 no son iguales.")
    return errores

# Función para validar la estructura del registro
def validar_registro_tipo1(registro):
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "0000")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "0000":
        errores.append("F_TIPO_REG no es '0000' o tiene una longitud incorrecta.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "00")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "00":
        errores.append("F_SUBTIPO_REG no es '00' o tiene una longitud incorrecta.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "01")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "01":
        errores.append("F_VERSION_REG no es '01' o tiene una longitud incorrecta.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")

    return errores

def validar_registro_tipo2(registro):
    global F350_ID_TIPO_DOCTO_GLOBAL
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "0350")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "0350":
        errores.append("F_TIPO_REG no es '0350' o tiene una longitud incorrecta.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "00")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "00":
        errores.append("F_SUBTIPO_REG no es '00' o tiene una longitud incorrecta.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "02")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "02":
        errores.append("F_VERSION_REG no es '02' o tiene una longitud incorrecta.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")
    
    # Validación de F_CONSEC_AUTO_REG (posición 19, longitud 1, debe ser "0" o "1")
    F_CONSEC_AUTO_REG = registro[18:19]
    if F_CONSEC_AUTO_REG not in ["0", "1"]:
        errores.append("F_CONSEC_AUTO_REG no es '0' o '1'.")
    
    # Validación de F350_ID_CO (posición 20-22, longitud 3, puede ser "01 " o espacios en blanco)
    F350_ID_CO = registro[19:22].replace("_", " ")
    if F350_ID_CO not in ["01 ", "   "]:
        errores.append("F350_ID_CO debe ser '01 ' o espacios en blanco.")
    
    # Validación de F350_ID_TIPO_DOCTO (posición 23-25, longitud 3, valor específico)
    valid_tipos = ["PL ", "NP ", "LC ", "PC ", "PS "]
    F350_ID_TIPO_DOCTO = registro[22:25].replace("_", " ")

    if F350_ID_TIPO_DOCTO_GLOBAL is None:
        F350_ID_TIPO_DOCTO_GLOBAL = F350_ID_TIPO_DOCTO
    elif F350_ID_TIPO_DOCTO != F350_ID_TIPO_DOCTO_GLOBAL:
        errores.append(f"F350_ID_TIPO_DOCTO {F350_ID_TIPO_DOCTO} no coincide con el valor esperado {F350_ID_TIPO_DOCTO_GLOBAL}.")
    
    if F350_ID_TIPO_DOCTO not in valid_tipos:
        errores.append("F350_ID_TIPO_DOCTO debe ser un valor válido: PL , NP , LC , PC , PS .")
   
        # Validación de F350_CONSEC_DOCTO (posición 26-33, longitud 8, debe ser numérico)
    F350_CONSEC_DOCTO = registro[25:33]
    if not F350_CONSEC_DOCTO.isdigit() or len(F350_CONSEC_DOCTO) != 8:
        errores.append("F350_CONSEC_DOCTO no es un número válido o no tiene 8 caracteres.")
    
    # Validación de F350_FECHA (posición 34-41, longitud 8, formato AAAAMMDD)
    F350_FECHA = registro[33:41]
    if not (len(F350_FECHA) == 8 and F350_FECHA.isdigit()):
        errores.append("F350_FECHA no tiene el formato AAAAMMDD.")
    
    # Validación de F350_ID_TERCERO (posición 42-56, longitud 15, puede ser espacios en blanco)
    F350_ID_TERCERO = registro[41:56]
    if not F350_ID_TERCERO.isspace() and len(F350_ID_TERCERO) != 15:
        errores.append("F350_ID_TERCERO debe tener 15 caracteres o estar en blanco.")
    
    # Validación de F350_ID_CLASE_DOCTO (posición 57-61, longitud 5, valor fijo = "00030")
    F350_ID_CLASE_DOCTO = registro[56:61]
    if F350_ID_CLASE_DOCTO != "00030":
        errores.append("F350_ID_CLASE_DOCTO no es '00030'.")
    
    # Validación de F350_IND_ESTADO (posición 62, longitud 1, valor 0, 1 o 2)
    F350_IND_ESTADO = registro[61:62]
    if F350_IND_ESTADO not in ["0", "1", "2"]:
        errores.append("F350_IND_ESTADO debe ser '0', '1' o '2'.")
    
    # Validación de F350_IND_IMPRESION (posición 63, longitud 1, valor 0 o 1)
    F350_IND_IMPRESION = registro[62:63]
    if F350_IND_IMPRESION not in ["0", "1"]:
        errores.append("F350_IND_IMPRESION debe ser '0' o '1'.")
    
    # Validación de F350_NOTAS (posición 64-318, longitud 255, contenido opcional)
    F350_NOTAS = registro[63:318]
    if len(F350_NOTAS) > 255:
        errores.append("F350_NOTAS tiene más de 255 caracteres.")
    
    # Validación de F350_ID_MANDATO (posición 319-333, longitud 15, puede estar en blanco)
    F350_ID_MANDATO = registro[318:333]
    if not F350_ID_MANDATO.strip() and len(F350_ID_MANDATO.strip()) != 0 and len(F350_ID_MANDATO) != 15:
        errores.append("F350_ID_MANDATO debe tener 15 caracteres o estar en blanco.")
            
    return errores

def validar_registro_tipo3(registro):
    global F350_ID_TIPO_DOCTO_GLOBAL
    global valores_DB_CR
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "0351")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "0351":
        errores.append("F_TIPO_REG no es '0351' o tiene una longitud incorrecta.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "00")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "00":
        errores.append("F_SUBTIPO_REG no es '00' o tiene una longitud incorrecta.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "04")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "04":
        errores.append("F_VERSION_REG no es '04' o tiene una longitud incorrecta.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")
    
    # Validación de F350_ID_CO (posición 19-21, longitud 3, puede ser "01 " o espacios en blanco)
    F350_ID_CO = registro[18:21].replace("_", " ")
    if F350_ID_CO not in ["01 ", "   "]:
        errores.append("F350_ID_CO debe ser '01 ' o espacios en blanco.")
    
    # Validación de F350_ID_TIPO_DOCTO (posición 22-24, longitud 3, valores específicos)
    valid_tipos = ["PL ", "NP ", "LC ", "PC ", "PS "]
    F350_ID_TIPO_DOCTO = registro[21:24].replace("_", " ")
    
    if F350_ID_TIPO_DOCTO_GLOBAL is None:
        F350_ID_TIPO_DOCTO_GLOBAL = F350_ID_TIPO_DOCTO
    elif F350_ID_TIPO_DOCTO != F350_ID_TIPO_DOCTO_GLOBAL:
        errores.append(f"F350_ID_TIPO_DOCTO {F350_ID_TIPO_DOCTO} no coincide con el valor esperado {F350_ID_TIPO_DOCTO_GLOBAL}.")
    
    if F350_ID_TIPO_DOCTO not in valid_tipos:
        errores.append("F350_ID_TIPO_DOCTO debe ser un valor válido: PL , NP , LC , PC , PS .")
    
    # Validación de F350_CONSEC_DOCTO (posición 25-32, longitud 8, debe ser numérico)
    F350_CONSEC_DOCTO = registro[24:32]
    if not F350_CONSEC_DOCTO.isdigit() or len(F350_CONSEC_DOCTO) != 8:
        errores.append("F350_CONSEC_DOCTO no es un número válido o no tiene 8 caracteres.")
    
    # Validación de F351_ID_AUXILIAR (posición 33-52, longitud 20, debe contener caracteres válidos o espacios en blanco)
    F351_ID_AUXILIAR = registro[32:52].replace("_", " ")
    if not F351_ID_AUXILIAR.replace(" ", "").isalnum() and not F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR contiene caracteres no permitidos o no tiene la longitud correcta.")
    
    # Validación de F351_ID_TERCERO (posición 53-67, longitud 15, puede ser espacios en blanco)
    F351_ID_TERCERO = registro[52:67].replace("_", " ")
    if not F351_ID_TERCERO.isspace() and len(F351_ID_TERCERO) != 15:
        errores.append("F351_ID_TERCERO debe tener 15 caracteres o estar en blanco.")
    
    # Validación adicional para F351_ID_TERCERO cuando auxiliar es 25 o 23 y no es uno de los valores permitidos
    if F351_ID_AUXILIAR.startswith(("25", "23")) and F351_ID_AUXILIAR.strip() != "25050101" and F350_ID_TIPO_DOCTO == "PS ":
        if F351_ID_TERCERO.strip() == "" or F351_ID_TERCERO.strip() not in valores_validos_tercero_ps:
            errores.append("F351_ID_TERCERO no debe estar en blanco o ser uno de los valores permitidos cuando inician por 25 o 23.")
        
    # Regla: Si F351_ID_AUXILIAR inicia con "5", F351_ID_TERCERO no puede ser solo de espacios
    if F351_ID_AUXILIAR.startswith("5") and F351_ID_TERCERO.isspace():
        errores.append("F351_ID_TERCERO no puede estar en blanco si F351_ID_AUXILIAR inicia con '5'.")
    
    # Validación de cuentas por cobrar que debe ir con otro documento
    if  F351_ID_AUXILIAR.startswith("136503"):
        errores.append("F351_ID_AUXILIAR que inicia con 136503 debe ir con otro documento.")
          
    # Regla: F351_ID_AUXILIAR no puede ser solo de espacios
    if F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR no puede estar en blanco.")
        
    # Validación de F351_ID_CO_MOV (posición 68-70, longitud 3, puede ser "01 " o espacios en blanco)
    F351_ID_CO_MOV = registro[67:70].replace("_", " ")
    if F351_ID_CO_MOV not in ["01 ", "   "]:
        errores.append("F351_ID_CO_MOV debe ser '01 ' o espacios en blanco.")
    
    # Validación de F351_ID_UN (posición 71-90, longitud 20, puede ser espacios en blanco)
    F351_ID_UN = registro[70:90].replace("_", " ")
    if not F351_ID_UN.isspace() and len(F351_ID_UN) != 20:
        errores.append("F351_ID_UN debe tener 20 caracteres o estar en blanco.")
    
    # Validación de F351_ID_CCOSTO (posición 91-105, longitud 15, puede ser espacios en blanco)
    F351_ID_CCOSTO = registro[90:105].replace("_", " ")
    if not F351_ID_CCOSTO.isspace() and len(F351_ID_CCOSTO) != 15:
        errores.append("F351_ID_CCOSTO debe tener 15 caracteres o estar en blanco.")

    # Validación adicional para unidad cuando la cuenta es 52050437
    if F351_ID_AUXILIAR.strip() == "52050437" and F351_ID_UN.strip() != "02":
        errores.append("F351_ID_UN no es 02")

    # Regla: Si F351_ID_AUXILIAR inicia con "25" o "23", F351_ID_UN y F351_ID_CCOSTO deben estar en blanco
    if F351_ID_AUXILIAR.startswith(("25", "23")):
        if F351_ID_UN.strip() or F351_ID_CCOSTO.strip():
            errores.append("Si F351_ID_AUXILIAR inicia con '25' o '23', F351_ID_UN y F351_ID_CCOSTO deben estar en blanco.")
    
    # Regla: Si F351_ID_AUXILIAR inicia con "5", F351_ID_UN y F351_ID_CCOSTO no deben estar en blanco
    if F351_ID_AUXILIAR.startswith(("5")):
        if not F351_ID_UN.strip() or not F351_ID_CCOSTO.strip():
            errores.append("Si F351_ID_AUXILIAR inicia con '5', F351_ID_UN y F351_ID_CCOSTO NO deben estar en blanco.")

   # Regla: Si F351_ID_CCOSTO inicia con "CD" o "CA" y F351_ID_AUXILIAR inicia con "5"
    if F351_ID_CCOSTO.startswith(("CD", "CA")) and F351_ID_AUXILIAR.startswith("5"):
    # Si F351_ID_CCOSTO inicia con "CD", F351_ID_AUXILIAR debe iniciar con "51"
        if F351_ID_CCOSTO.startswith("CD") and not F351_ID_AUXILIAR.startswith("51"):
            errores.append("F351_ID_AUXILIAR debe iniciar con '51' cuando F351_ID_CCOSTO inicia con 'CD'.")
    
    # Si F351_ID_CCOSTO inicia con "CA", F351_ID_AUXILIAR debe iniciar con "52" o "50"
        elif F351_ID_CCOSTO.startswith("CA") and not (F351_ID_AUXILIAR.startswith("52") or F351_ID_AUXILIAR.startswith("50")):
            errores.append("F351_ID_AUXILIAR debe iniciar con '52' o '50' cuando F351_ID_CCOSTO inicia con 'CA'.")
        
    # Validación de F351_VALOR_DB (posición 116-136, formato numérico con signo y decimales)
    F351_VALOR_DB = registro[115:136]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB):
        errores.append("F351_VALOR_DB no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR (posición 137-157, formato numérico con signo y decimales)
    F351_VALOR_CR = registro[136:157]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR):
        errores.append("F351_VALOR_CR no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Regla: Si F351_VALOR_DB tiene un valor, F351_VALOR_CR debe estar en 0 y viceversa
    if float(F351_VALOR_DB) != 0 and float(F351_VALOR_CR) != 0:
        errores.append("Si F351_VALOR_DB tiene un valor, F351_VALOR_CR debe estar en 0 y viceversa.")
    
    # Validación de F351_VALOR_DB_ALT (posición 158-178, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT = registro[157:178]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT):
        errores.append("F351_VALOR_DB_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT (posición 179-199, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT = registro[178:199]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT):
        errores.append("F351_VALOR_CR_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE (posición 200-220, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE = registro[199:220]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE):
        errores.append("F351_BASE_GRAVABLE no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB2 (posición 221-241, formato numérico con signo y decimales)
    F351_VALOR_DB2 = registro[220:241]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB2):
        errores.append("F351_VALOR_DB2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR2 (posición 242-262, formato numérico con signo y decimales)
    F351_VALOR_CR2 = registro[241:262]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR2):
        errores.append("F351_VALOR_CR2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB_ALT2 (posición 263-283, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT2 = registro[262:283]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT2):
        errores.append("F351_VALOR_DB_ALT2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT2 (posición 284-304, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT2 = registro[283:304]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT2):
        errores.append("F351_VALOR_CR_ALT2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE2 (posición 305-325, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE2 = registro[304:325]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE2):
        errores.append("F351_BASE_GRAVABLE2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB3 (posición 325-346, formato numérico con signo y decimales)
    F351_VALOR_DB3 = registro[325:346]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB3):
        errores.append("F351_VALOR_DB3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR3 (posición 346-367, formato numérico con signo y decimales)
    F351_VALOR_CR3 = registro[346:367]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR3):
        errores.append("F351_VALOR_CR3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB_ALT3 (posición 367-388, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT3 = registro[367:388]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT3):
        errores.append("F351_VALOR_DB_ALT3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT3 (posición 388-409, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT3 = registro[388:409]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT3):
        errores.append("F351_VALOR_CR_ALT3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE3 (posición 409-430, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE3 = registro[409:430]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE3):
        errores.append("F351_BASE_GRAVABLE3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")    
    
    # Validación de F351_VALOR_IMP_ASUM (posición 430-451, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM = registro[430:451]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM):
        errores.append("F351_VALOR_IMP_ASUM no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")    
    
    # Validación de F351_VALOR_IMP_ASUM2 (posición 451-472, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM2 = registro[451:472]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM2):
        errores.append("F351_VALOR_IMP_ASUM2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")    
    
    # Validación de F351_VALOR_IMP_ASUM3 (posición 472-493, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM3 = registro[472:493]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM3):
        errores.append("F351_VALOR_IMP_ASUM3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).") 
    
    # Validación de F351_DOCTO_BANCO (posición 493-495, longitud 2, puede ser espacios en blanco)
    F351_DOCTO_BANCO = registro[493:495].replace("_", " ")
    if not F351_DOCTO_BANCO.isspace() and len(F351_DOCTO_BANCO) != 2:
        errores.append("F351_DOCTO_BANCO debe tener 20 caracteres o estar en blanco.")
    
    # Validación de F351_NRO_DOCTO_BANCO (posición 495-503, longitud 8, valor fijo = "00000000")
    F351_NRO_DOCTO_BANCO = registro[495:503]
    if F351_NRO_DOCTO_BANCO != "00000000":
        errores.append("F351_NRO_DOCTO_BANCO no es '00000000' o tiene una longitud incorrecta.")
    
    # Validación de F351_NRO_ALT_DOCTO_BANCO (posición 503-533, longitud 2, puede ser espacios en blanco)
    F351_NRO_ALT_DOCTO_BANCO = registro[503:533].replace("_", " ")
    if not F351_NRO_ALT_DOCTO_BANCO.isspace() and len(F351_NRO_ALT_DOCTO_BANCO) != 30:
        errores.append("F351_NRO_ALT_DOCTO_BANCO debe tener 30 caracteres o estar en blanco.")
    
    # Validación de F351_NOTAS (posición 534-788, longitud 255, contenido opcional)
    F351_NOTAS = registro[533:788]
    if len(F351_NOTAS) > 255:
        errores.append("F351_NOTAS tiene más de 255 caracteres.")
    
    # Almacenar los valores en la variable global
    valores_DB_CR['F351_VALOR_DB'].append(F351_VALOR_DB)
    valores_DB_CR['F351_VALOR_DB2'].append(F351_VALOR_DB2)
    valores_DB_CR['F351_VALOR_DB3'].append(F351_VALOR_DB3)
    valores_DB_CR['F351_VALOR_CR'].append(F351_VALOR_CR)
    valores_DB_CR['F351_VALOR_CR2'].append(F351_VALOR_CR2)
    valores_DB_CR['F351_VALOR_CR3'].append(F351_VALOR_CR3)
    
    # Validar que F351_VALOR_DB, F351_VALOR_DB2 y F351_VALOR_DB3 sean iguales
    errores += validar_igualdad_valores_DB(F351_VALOR_DB, F351_VALOR_DB2, F351_VALOR_DB3)
        
    # Validar que F351_VALOR_CR, F351_VALOR_CR2 y F351_VALOR_CR3 sean iguales
    errores += validar_igualdad_valores_CR(F351_VALOR_CR, F351_VALOR_CR2, F351_VALOR_CR3)
    
    # Regla: F351_VALOR_DB no debe ser igual a F351_VALOR_CR
    if F351_VALOR_DB == F351_VALOR_CR:
        errores.append("F351_VALOR_DB no debe ser igual a F351_VALOR_CR en la misma fila.")

    # Regla: F351_VALOR_DB2 no debe ser igual a F351_VALOR_CR2
    if F351_VALOR_DB2 == F351_VALOR_CR2:
        errores.append("F351_VALOR_DB2 no debe ser igual a F351_VALOR_CR2 en la misma fila.")

    # Regla: F351_VALOR_DB3 no debe ser igual a F351_VALOR_CR3
    if F351_VALOR_DB3 == F351_VALOR_CR3:
        errores.append("F351_VALOR_DB3 no debe ser igual a F351_VALOR_CR3 en la misma fila.")
        
    return errores

def validar_registro_tipo4(registro):
    global F350_ID_TIPO_DOCTO_GLOBAL
    global suma_valor_db_tipo4, suma_valor_cr_tipo4
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "0351")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "0351":
        errores.append("F_TIPO_REG no es '0351' o tiene una longitud incorrecta.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "01")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "01":
        errores.append("F_SUBTIPO_REG no es '01' o tiene una longitud incorrecta.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "02")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "02":
        errores.append("F_VERSION_REG no es '02' o tiene una longitud incorrecta.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")
    
    # Validación de F350_ID_CO (posición 19-21, longitud 3, puede ser "01 " o espacios en blanco)
    F350_ID_CO = registro[18:21].replace("_", " ")
    if F350_ID_CO not in ["01 ", "   "]:
        errores.append("F350_ID_CO debe ser '01 ' o espacios en blanco.")
    
    # Validación de F350_ID_TIPO_DOCTO (posición 22-24, longitud 3, valores específicos)
    valid_tipos = ["PL ", "NP ", "LC ", "PC ", "PS "]
    F350_ID_TIPO_DOCTO = registro[21:24].replace("_", " ")
    
    if F350_ID_TIPO_DOCTO_GLOBAL is None:
        F350_ID_TIPO_DOCTO_GLOBAL = F350_ID_TIPO_DOCTO
    elif F350_ID_TIPO_DOCTO != F350_ID_TIPO_DOCTO_GLOBAL:
        errores.append(f"F350_ID_TIPO_DOCTO {F350_ID_TIPO_DOCTO} no coincide con el valor esperado {F350_ID_TIPO_DOCTO_GLOBAL}.")
    
    if F350_ID_TIPO_DOCTO not in valid_tipos:
        errores.append("F350_ID_TIPO_DOCTO debe ser un valor válido: PL , NP , LC , PC , PS .")    
    
    # Validación de F350_CONSEC_DOCTO (posición 25-32, longitud 8, debe ser numérico)
    F350_CONSEC_DOCTO = registro[24:32]
    if not F350_CONSEC_DOCTO.isdigit() or len(F350_CONSEC_DOCTO) != 8:
        errores.append("F350_CONSEC_DOCTO no es un número válido o no tiene 8 caracteres.")
    
    # Validación de F351_ID_AUXILIAR (posición 33-52, longitud 20, debe contener caracteres válidos o espacios en blanco)
    F351_ID_AUXILIAR = registro[32:52].replace("_", " ")
    if not F351_ID_AUXILIAR.replace(" ", "").isalnum() and not F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR contiene caracteres no permitidos o no tiene la longitud correcta.")
    
    # Validación de F351_ID_TERCERO (posición 53-67, longitud 15, puede ser opcional)
    F351_ID_TERCERO = registro[52:67].replace("_", " ")
    if not F351_ID_TERCERO.isspace() and len(F351_ID_TERCERO) != 15:
        errores.append("F351_ID_TERCERO debe tener 15 caracteres o estar en blanco.")
    
    # Regla: Si F351_ID_AUXILIAR inicia con "5", F351_ID_TERCERO no puede ser solo de espacios
    if F351_ID_AUXILIAR.startswith("5") and F351_ID_TERCERO.isspace():
        errores.append("F351_ID_TERCERO no puede estar en blanco si F351_ID_AUXILIAR inicia con '5'.")

    # Regla: F351_ID_AUXILIAR no puede ser solo de espacios
    if F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR no puede estar en blanco.")
    
    # Validación de F351_ID_CO_MOV (posición 68-70, longitud 3, puede ser "01 " o espacios en blanco)
    F351_ID_CO_MOV = registro[67:70].replace("_", " ")
    if F351_ID_CO_MOV not in ["01 ", "   "]:
        errores.append("F351_ID_CO_MOV debe ser '01 ' o espacios en blanco.")
    
    # Validación de F351_ID_UN (posición 71-90, longitud 20, puede ser espacios en blanco)
    F351_ID_UN = registro[70:90].replace("_", " ")
    if not F351_ID_UN.isspace() and len(F351_ID_UN) != 20:
        errores.append("F351_ID_UN debe tener 20 caracteres o estar en blanco.")
    
    # Validación de F351_ID_CCOSTO (posición 91-105, longitud 15, puede ser espacios en blanco)
    F351_ID_CCOSTO = registro[90:105].replace("_", " ")
    if not F351_ID_CCOSTO.isspace() and len(F351_ID_CCOSTO) != 15:
        errores.append("F351_ID_CCOSTO debe tener 15 caracteres o estar en blanco.")
    
    # Validación de F351_VALOR_DB (posición 106-126, formato numérico con signo y decimales)
    F351_VALOR_DB = registro[105:126]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB):
        errores.append("F351_VALOR_DB no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    else:
        suma_valor_db_tipo4 += float(F351_VALOR_DB)
    
    # Validación de F351_VALOR_CR (posición 127-147, formato numérico con signo y decimales)
    F351_VALOR_CR = registro[126:147]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR):
        errores.append("F351_VALOR_CR no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    else:
        suma_valor_cr_tipo4 += float(F351_VALOR_CR)
    
    # Validación de F351_VALOR_DB_ALT (posición 148-168, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT = registro[147:168]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT):
        errores.append("F351_VALOR_DB_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT (posición 169-189, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT = registro[168:189]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT):
        errores.append("F351_VALOR_CR_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_NOTAS (posición 190-444, longitud 255, contenido opcional)
    F351_NOTAS = registro[189:444]
    if len(F351_NOTAS) > 255:
        errores.append("F351_NOTAS tiene más de 255 caracteres.")
    
    # Validación de F353_ID_SUCURSAL (posición 445-447, longitud 3, debe ser numérico)
    F353_ID_SUCURSAL = registro[444:447]
    if not F353_ID_SUCURSAL.isdigit() or len(F353_ID_SUCURSAL) != 3:
        errores.append("F353_ID_SUCURSAL no es un código de sucursal válido o no tiene 3 caracteres.")
    
    # Validación de F353_ID_TIPO_DOCTO_CRUCE (posición 448-450, longitud 3, debe ser alfanumérico)
    F353_ID_TIPO_DOCTO_CRUCE = registro[447:450]
    if not F353_ID_TIPO_DOCTO_CRUCE.isalnum() or len(F353_ID_TIPO_DOCTO_CRUCE) != 3:
        errores.append("F353_ID_TIPO_DOCTO_CRUCE no es válido o no tiene 3 caracteres.")
    
    # Validación de F353_CONSEC_DOCTO_CRUCE (posición 451-458, longitud 8, debe ser numérico)
    F353_CONSEC_DOCTO_CRUCE = registro[450:458]
    if not F353_CONSEC_DOCTO_CRUCE.isdigit() or len(F353_CONSEC_DOCTO_CRUCE) != 8:
        errores.append("F353_CONSEC_DOCTO_CRUCE no es un número válido o no tiene 8 caracteres.")
    
    # Validación de F353_NRO_CUOTA_CRUCE (posición 459-461, longitud 3, debe ser numérico)
    F353_NRO_CUOTA_CRUCE = registro[458:461]
    if not F353_NRO_CUOTA_CRUCE.isdigit() or len(F353_NRO_CUOTA_CRUCE) != 3:
        errores.append("F353_NRO_CUOTA_CRUCE no es un número válido o no tiene 3 caracteres.")
    
    # Validación de F353_FECHA_VCTO (posición 462-469, longitud 8, formato AAAAMMDD)
    F353_FECHA_VCTO = registro[461:469]
    if not re.match(r"\d{8}", F353_FECHA_VCTO):
        errores.append("F353_FECHA_VCTO no tiene el formato AAAAMMDD.")
    
    # Validación de F353_FECHA_DSCTO_PP (posición 470-477, longitud 8, formato AAAAMMDD)
    F353_FECHA_DSCTO_PP = registro[469:477]
    if not re.match(r"\d{8}", F353_FECHA_DSCTO_PP):
        errores.append("F353_FECHA_DSCTO_PP no tiene el formato AAAAMMDD.")
    
    # Validación de F353_VLR_DSCTO_PP (posición 478-498, formato numérico con signo y decimales)
    F353_VLR_DSCTO_PP = registro[477:498]
    if not re.match(r"[+-]\d{15}\.\d{4}", F353_VLR_DSCTO_PP):
        errores.append("F353_VLR_DSCTO_PP no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_APLICADO_PP (posición 499-519, formato numérico con signo y decimales)
    F354_VALOR_APLICADO_PP = registro[498:519]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_APLICADO_PP):
        errores.append("F354_VALOR_APLICADO_PP no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_APLICADO_PP_ALT (posición 520-540, formato numérico con signo y decimales)
    F354_VALOR_APLICADO_PP_ALT = registro[519:540]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_APLICADO_PP_ALT):
        errores.append("F354_VALOR_APLICADO_PP_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_APROVECHA (posición 541-561, formato numérico con signo y decimales)
    F354_VALOR_APROVECHA = registro[540:561]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_APROVECHA):
        errores.append("F354_VALOR_APROVECHA no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_APROVECHA_ALT (posición 562-582, formato numérico con signo y decimales)
    F354_VALOR_APROVECHA_ALT = registro[561:582]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_APROVECHA_ALT):
        errores.append("F354_VALOR_APROVECHA_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_RETENCION (posición 583-603, formato numérico con signo y decimales)
    F354_VALOR_RETENCION = registro[582:603]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_RETENCION):
        errores.append("F354_VALOR_RETENCION no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_VALOR_RETENCION_ALT (posición 604-624, formato numérico con signo y decimales)
    F354_VALOR_RETENCION_ALT = registro[603:624]
    if not re.match(r"[+-]\d{15}\.\d{4}", F354_VALOR_RETENCION_ALT):
        errores.append("F354_VALOR_RETENCION_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F354_TERCERO_VEND (posición 625-639, longitud 15, puede ser espacios en blanco)
    F354_TERCERO_VEND = registro[624:639].replace("_", " ")
    if not F354_TERCERO_VEND.isspace() and len(F354_TERCERO_VEND) != 15:
        errores.append("F354_TERCERO_VEND debe tener 15 caracteres o estar en blanco.")
    
    # Validación de F353_FECHA_DOCTO_CRUCE (posición 640-647, longitud 8, formato AAAAMMDD)
    F353_FECHA_DOCTO_CRUCE = registro[639:647]
    if not re.match(r"\d{8}", F353_FECHA_DOCTO_CRUCE):
        errores.append("F353_FECHA_DOCTO_CRUCE no tiene el formato AAAAMMDD.")
    
    # Validación de F353_FECHA_RADICACION (posición 648-655, longitud 8, formato AAAAMMDD)
    F353_FECHA_RADICACION = registro[647:655]
    if not re.match(r"\d{8}", F353_FECHA_RADICACION):
        errores.append("F353_FECHA_RADICACION no tiene el formato AAAAMMDD.")
    
    # Validación de F354_NOTAS (posición 656-910, longitud 255, contenido opcional)
    F354_NOTAS = registro[655:910]
    if len(F354_NOTAS) > 255:
        errores.append("F354_NOTAS tiene más de 255 caracteres.")
    
    # Almacenar los valores en la variable global
    #valores_DB_CR['F351_VALOR_DB'].append(F351_VALOR_DB)
    #valores_DB_CR['F351_VALOR_CR'].append(F351_VALOR_CR)
    
    # Regla: F351_VALOR_DB no debe ser igual a F351_VALOR_CR
    if F351_VALOR_DB == F351_VALOR_CR:
        errores.append("F351_VALOR_DB no debe ser igual a F351_VALOR_CR en la misma fila.")

    return errores

def validar_registro_tipo5(registro):
    global F350_ID_TIPO_DOCTO_GLOBAL
    global valores_DB_CR
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "0351")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "0351":
        errores.append("F_TIPO_REG no es '0351' o tiene una longitud incorrecta.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "00")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "00":
        errores.append("F_SUBTIPO_REG no es '00' o tiene una longitud incorrecta.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "04")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "04":
        errores.append("F_VERSION_REG no es '04' o tiene una longitud incorrecta.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")
    
    # Validación de F350_ID_CO (posición 19-21, longitud 3, puede ser "01 " o espacios en blanco)
    F350_ID_CO = registro[18:21].replace("_", " ")
    if F350_ID_CO not in ["01 ", "   "]:
        errores.append("F350_ID_CO debe ser '01 ' o espacios en blanco.")
    
    # Validación de F350_ID_TIPO_DOCTO (posición 22-24, longitud 3, valores específicos)
    valid_tipos = ["PL ", "NP ", "LC ", "PC ", "PS "]
    F350_ID_TIPO_DOCTO = registro[21:24].replace("_", " ")
    
    if F350_ID_TIPO_DOCTO_GLOBAL is None:
        F350_ID_TIPO_DOCTO_GLOBAL = F350_ID_TIPO_DOCTO
    elif F350_ID_TIPO_DOCTO != F350_ID_TIPO_DOCTO_GLOBAL:
        errores.append(f"F350_ID_TIPO_DOCTO {F350_ID_TIPO_DOCTO} no coincide con el valor esperado {F350_ID_TIPO_DOCTO_GLOBAL}.")
    
    if F350_ID_TIPO_DOCTO not in valid_tipos:
        errores.append("F350_ID_TIPO_DOCTO debe ser un valor válido: PL , NP , LC , PC , PS .")    
        
    # Validación de F350_CONSEC_DOCTO (posición 25-32, longitud 8, debe ser numérico)
    F350_CONSEC_DOCTO = registro[24:32]
    if not F350_CONSEC_DOCTO.isdigit() or len(F350_CONSEC_DOCTO) != 8:
        errores.append("F350_CONSEC_DOCTO no es un número válido o no tiene 8 caracteres.")
    
    # Validación de F351_ID_AUXILIAR (posición 33-52, longitud 20, debe contener caracteres válidos o espacios en blanco)
    F351_ID_AUXILIAR = registro[32:52].replace("_", " ")
    if not F351_ID_AUXILIAR.replace(" ", "").isalnum() and not F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR contiene caracteres no permitidos o no tiene la longitud correcta.")
    
    # Validación de F351_ID_TERCERO (posición 53-67, longitud 15, puede ser opcional)
    F351_ID_TERCERO = registro[52:67].replace("_", " ")
    if not F351_ID_TERCERO.isspace() and len(F351_ID_TERCERO) != 15:
        errores.append("F351_ID_TERCERO debe tener 15 caracteres o estar en blanco.")
   
    # Regla: Si F351_ID_AUXILIAR inicia con "5", F351_ID_TERCERO no puede ser solo de espacios
    if F351_ID_AUXILIAR.startswith("5") and F351_ID_TERCERO.isspace():
        errores.append("F351_ID_TERCERO no puede estar en blanco si F351_ID_AUXILIAR inicia con '5'.")

    # Regla: F351_ID_AUXILIAR no puede ser solo de espacios
    if F351_ID_AUXILIAR.isspace():
        errores.append("F351_ID_AUXILIAR no puede estar en blanco.")
       
    # Validación de F351_ID_CO_MOV (posición 68-70, longitud 3, puede ser "01 " o espacios en blanco)
    F351_ID_CO_MOV = registro[67:70].replace("_", " ")
    if F351_ID_CO_MOV not in ["01 ", "   "]:
        errores.append("F351_ID_CO_MOV debe ser '01 ' o espacios en blanco.")
    
    # Validación de F351_ID_UN (posición 71-90, longitud 20, puede ser espacios en blanco)
    F351_ID_UN = registro[70:90].replace("_", " ")
    if not F351_ID_UN.isspace() and len(F351_ID_UN) != 20:
        errores.append("F351_ID_UN debe tener 20 caracteres o estar en blanco.")
    
    # Validación de F351_ID_CCOSTO (posición 91-105, longitud 15, puede ser espacios en blanco)
    F351_ID_CCOSTO = registro[90:105].replace("_", " ")
    if not F351_ID_CCOSTO.isspace() and len(F351_ID_CCOSTO) != 15:
        errores.append("F351_ID_CCOSTO debe tener 15 caracteres o estar en blanco.")
    
    # Regla: Si F351_ID_CCOSTO inicia con "CA", F351_ID_AUXILIAR debe iniciar con "52"
    if F351_ID_CCOSTO.startswith("CA") and not F351_ID_AUXILIAR.startswith("52"):
        errores.append("F351_ID_AUXILIAR debe iniciar con '52' cuando F351_ID_CCOSTO inicia con 'CA'.")

    # Regla: Si F351_ID_CCOSTO inicia con "CD", F351_ID_AUXILIAR debe iniciar con "51"
    if F351_ID_CCOSTO.startswith("CD") and not F351_ID_AUXILIAR.startswith("51"):
        errores.append("F351_ID_AUXILIAR debe iniciar con '51' cuando F351_ID_CCOSTO inicia con 'CD'.")
    
    # Validación de F351_ID_FE (posición 106-115, longitud 10, puede ser espacios en blanco)
    F351_ID_FE = registro[105:115].replace("_", " ")
    if not F351_ID_FE.isspace() and len(F351_ID_FE) != 10:
        errores.append("F351_ID_FE debe tener 10 caracteres o estar en blanco.")
    
    # Validación de F351_VALOR_DB (posición 116-136, formato numérico con signo y decimales)
    F351_VALOR_DB = registro[115:136]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB):
        errores.append("F351_VALOR_DB no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR (posición 137-157, formato numérico con signo y decimales)
    F351_VALOR_CR = registro[136:157]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR):
        errores.append("F351_VALOR_CR no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB_ALT (posición 158-178, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT = registro[157:178]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT):
        errores.append("F351_VALOR_DB_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT (posición 179-199, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT = registro[178:199]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT):
        errores.append("F351_VALOR_CR_ALT no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE (posición 200-220, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE = registro[199:220]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE):
        errores.append("F351_BASE_GRAVABLE no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB2 (posición 221-241, formato numérico con signo y decimales)
    F351_VALOR_DB2 = registro[220:241]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB2):
        errores.append("F351_VALOR_DB2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR2 (posición 242-262, formato numérico con signo y decimales)
    F351_VALOR_CR2 = registro[241:262]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR2):
        errores.append("F351_VALOR_CR2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB_ALT2 (posición 263-283, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT2 = registro[262:283]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT2):
        errores.append("F351_VALOR_DB_ALT2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT2 (posición 284-304, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT2 = registro[283:304]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT2):
        errores.append("F351_VALOR_CR_ALT2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE2 (posición 305-325, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE2 = registro[304:325]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE2):
        errores.append("F351_BASE_GRAVABLE2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB3 (posición 325-346, formato numérico con signo y decimales)
    F351_VALOR_DB3 = registro[325:346]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB3):
        errores.append("F351_VALOR_DB3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR3 (posición 346-367, formato numérico con signo y decimales)
    F351_VALOR_CR3 = registro[346:367]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR3):
        errores.append("F351_VALOR_CR3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_DB_ALT3 (posición 367-388, formato numérico con signo y decimales)
    F351_VALOR_DB_ALT3 = registro[367:388]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_DB_ALT3):
        errores.append("F351_VALOR_DB_ALT3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_VALOR_CR_ALT3 (posición 388-409, formato numérico con signo y decimales)
    F351_VALOR_CR_ALT2 = registro[388:409]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_CR_ALT2):
        errores.append("F351_VALOR_CR_ALT2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
    
    # Validación de F351_BASE_GRAVABLE3 (posición 409-430, formato numérico con signo y decimales)
    F351_BASE_GRAVABLE2 = registro[409:430]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_BASE_GRAVABLE2):
        errores.append("F351_BASE_GRAVABLE2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")
   
    # Validación de F351_VALOR_IMP_ASUM (posición 430-451, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM = registro[430:451]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM):
        errores.append("F351_VALOR_IMP_ASUM no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")    
    
    # Validación de F351_VALOR_IMP_ASUM2 (posición 451-472, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM2 = registro[451:472]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM2):
        errores.append("F351_VALOR_IMP_ASUM2 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).")    
    
    # Validación de F351_VALOR_IMP_ASUM3 (posición 472-493, formato numérico con signo y decimales)
    F351_VALOR_IMP_ASUM3 = registro[472:493]
    if not re.match(r"[+-]\d{15}\.\d{4}", F351_VALOR_IMP_ASUM3):
        errores.append("F351_VALOR_IMP_ASUM3 no tiene el formato correcto (signo + 15 enteros + punto + 4 decimales).") 
    
    # Validación de F351_DOCTO_BANCO (posición 493-495, longitud 2, puede ser espacios en blanco)
    F351_DOCTO_BANCO = registro[493:495].replace("_", " ")
    if not F351_DOCTO_BANCO.isspace() and len(F351_DOCTO_BANCO) != 2:
        errores.append("F351_DOCTO_BANCO debe tener 20 caracteres o estar en blanco.")
    
    # Validación de F351_NRO_DOCTO_BANCO (posición 495-503, longitud 8, valor fijo = "00000000")
    F351_NRO_DOCTO_BANCO = registro[495:503]
    if F351_NRO_DOCTO_BANCO != "00000000":
        errores.append("F351_NRO_DOCTO_BANCO no es '00000000' o tiene una longitud incorrecta.")
    
    # Validación de F351_NRO_ALT_DOCTO_BANCO (posición 503-533, longitud 2, puede ser espacios en blanco)
    F351_NRO_ALT_DOCTO_BANCO = registro[503:533].replace("_", " ")
    if not F351_NRO_ALT_DOCTO_BANCO.isspace() and len(F351_NRO_ALT_DOCTO_BANCO) != 30:
        errores.append("F351_NRO_ALT_DOCTO_BANCO debe tener 30 caracteres o estar en blanco.")
    
    # Validación de F351_NOTAS (posición 534-788, longitud 255, contenido opcional)
    F351_NOTAS = registro[533:788]
    if len(F351_NOTAS) > 255:
        errores.append("F351_NOTAS tiene más de 255 caracteres.")
   
   # Almacenar los valores en la variable global
    valores_DB_CR['F351_VALOR_DB'].append(F351_VALOR_DB)
    valores_DB_CR['F351_VALOR_DB2'].append(F351_VALOR_DB2)
    valores_DB_CR['F351_VALOR_DB3'].append(F351_VALOR_DB3)
    valores_DB_CR['F351_VALOR_CR'].append(F351_VALOR_CR)
    valores_DB_CR['F351_VALOR_CR2'].append(F351_VALOR_CR2)
    valores_DB_CR['F351_VALOR_CR3'].append(F351_VALOR_CR3)
   
    # Validar que F351_VALOR_CR, F351_VALOR_CR2 y F351_VALOR_CR3 sean iguales
    errores += validar_igualdad_valores_CR(F351_VALOR_CR, F351_VALOR_CR2, F351_VALOR_CR3)
    
    # Regla: F351_VALOR_DB no debe ser igual a F351_VALOR_CR
    if F351_VALOR_DB == F351_VALOR_CR:
        errores.append("F351_VALOR_DB no debe ser igual a F351_VALOR_CR en la misma fila.")

    # Regla: F351_VALOR_DB2 no debe ser igual a F351_VALOR_CR2
    if F351_VALOR_DB2 == F351_VALOR_CR2:
        errores.append("F351_VALOR_DB2 no debe ser igual a F351_VALOR_CR2 en la misma fila.")

    # Regla: F351_VALOR_DB3 no debe ser igual a F351_VALOR_CR3
    if F351_VALOR_DB3 == F351_VALOR_CR3:
        errores.append("F351_VALOR_DB3 no debe ser igual a F351_VALOR_CR3 en la misma fila.")

    return errores

def validar_registro_tipo6(registro):
    errores = []
    
    # Validación de F_NUMERO_REG (posición 1-7, longitud 7, debe ser numérico)
    F_NUMERO_REG = registro[0:7]
    errores += validar_consecutivo_F_NUMERO_REG(F_NUMERO_REG)  # Llamada a la función de consecutivo
    if not F_NUMERO_REG.isdigit() or len(F_NUMERO_REG) != 7:
        errores.append("F_NUMERO_REG no es un número válido o no tiene 7 caracteres.")
    
    # Validación de F_TIPO_REG (posición 8-11, longitud 4, valor fijo = "9999")
    F_TIPO_REG = registro[7:11]
    if F_TIPO_REG != "9999":
        errores.append("F_TIPO_REG no es '9999'.")
    
    # Validación de F_SUBTIPO_REG (posición 12-13, longitud 2, valor fijo = "00")
    F_SUBTIPO_REG = registro[11:13]
    if F_SUBTIPO_REG != "00":
        errores.append("F_SUBTIPO_REG no es '00'.")
    
    # Validación de F_VERSION_REG (posición 14-15, longitud 2, valor fijo = "01")
    F_VERSION_REG = registro[13:15]
    if F_VERSION_REG != "01":
        errores.append("F_VERSION_REG no es '01'.")
    
    # Validación de F_CIA (posición 16-18, longitud 3, debe ser numérico)
    F_CIA = registro[15:18]
    if not F_CIA.isdigit() or len(F_CIA) != 3:
        errores.append("F_CIA no es un código de compañía válido o no tiene 3 caracteres.")
        
    return errores

# Función principal para identificar el tipo de registro y validar en función del tipo
def validar_registro(registro):
    tipo_registro = registro[7:11]  # Posición donde está el tipo de registro
    
    if tipo_registro == "0000":
        return validar_registro_tipo1(registro)
    elif tipo_registro == "0350":
        return validar_registro_tipo2(registro)
    elif tipo_registro == "0351" and registro[11:13] == "00":
        return validar_registro_tipo3(registro)
    elif tipo_registro == "0351" and registro[11:13] == "01":
        return validar_registro_tipo4(registro)
    elif tipo_registro == "9999":
        return validar_registro_tipo6(registro)
    else:
        return [f"Tipo de registro no reconocido: {tipo_registro}"]
    
# Función para validar la codificación del archivo
def validar_codificacion(archivo):
    with open(archivo, 'rb') as file:
        rawdata = file.read()
    result = chardet.detect(rawdata)
    encoding = result['encoding']
    if encoding != 'ascii':
        return [f"Codificación del archivo no es ANSI , es {encoding}."]
    else:
        print("Validación 1: Codificación del archivo 'Cumple con ANSI'.")
    return []

# Función que se ejecuta al finalizar todas las validaciones
def validar_registros_tipo1(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo1(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con fila inicial.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en la fila inicial: {errores_totales}")
    return errores_totales

def validar_registros_tipo2(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo2(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con fila de encabezado.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en la fila del encabezado: {errores_totales}")
    return errores_totales

def validar_registros_tipo3(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo3(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con las filas del movimiento.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en las filas del movimiento: {errores_totales}")
    return errores_totales

def validar_registros_tipo4(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo4(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con las filas de cuenta por cobrar.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en las filas de la cuenta por cobrar: {errores_totales}")
    return errores_totales

def validar_registros_tipo5(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo5(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con la fila del gasto.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en la fila del gasto: {errores_totales}")
    return errores_totales

def validar_registros_tipo6(registros):
    errores_totales = []
    
    for registro in registros:
        errores = validar_registro_tipo6(registro)
        errores_totales.extend(errores)

    if not errores_totales:
        print("Validación 2: Cumple con la fila final.")  # Mensaje de validación exitosa
    else:
        print(f"Se encontraron errores en la fila final: {errores_totales}")
    return errores_totales

# Función para imprimir los valores finales
def imprimir_valores_finales():
    suma_valor_db = 0
    suma_valor_db2 = 0
    suma_valor_db3 = 0
    suma_valor_cr = 0
    suma_valor_cr2 = 0
    suma_valor_cr3 = 0

    # Encontrar la longitud mínima de las listas en el diccionario
    longitud_minima = min(len(valores_DB_CR['F351_VALOR_DB']),
                          len(valores_DB_CR['F351_VALOR_DB2']),
                          len(valores_DB_CR['F351_VALOR_DB3']),
                          len(valores_DB_CR['F351_VALOR_CR']),
                          len(valores_DB_CR['F351_VALOR_CR2']),
                          len(valores_DB_CR['F351_VALOR_CR3']))

    for i in range(longitud_minima):
        # Verificar que los valores no estén vacíos antes de convertirlos a float
        if valores_DB_CR['F351_VALOR_DB'][i] and valores_DB_CR['F351_VALOR_DB2'][i] and valores_DB_CR['F351_VALOR_DB3'][i] and valores_DB_CR['F351_VALOR_CR'][i] and valores_DB_CR['F351_VALOR_CR2'][i] and valores_DB_CR['F351_VALOR_CR3'][i]:
            valor_db = float(valores_DB_CR['F351_VALOR_DB'][i])
            valor_db2 = float(valores_DB_CR['F351_VALOR_DB2'][i])
            valor_db3 = float(valores_DB_CR['F351_VALOR_DB3'][i])
            valor_cr = float(valores_DB_CR['F351_VALOR_CR'][i])
            valor_cr2 = float(valores_DB_CR['F351_VALOR_CR2'][i])
            valor_cr3 = float(valores_DB_CR['F351_VALOR_CR3'][i])
            
            # Sumar los valores
            suma_valor_db += valor_db
            suma_valor_db2 += valor_db2
            suma_valor_db3 += valor_db3
            suma_valor_cr += valor_cr
            suma_valor_cr2 += valor_cr2
            suma_valor_cr3 += valor_cr3

    # Verificar si todos los valores están cuadrados
    if (suma_valor_db + suma_valor_db_tipo4 == suma_valor_cr + suma_valor_cr_tipo4 and
        suma_valor_db2 + suma_valor_db_tipo4 == suma_valor_cr2 + suma_valor_cr_tipo4 and
        suma_valor_db3 + suma_valor_db_tipo4 == suma_valor_cr3 + suma_valor_cr_tipo4):
            print("Validación 3: Están cuadrados los débitos y los créditos.")
            print(f"  Débitos: {locale.currency(suma_valor_db+suma_valor_db_tipo4, grouping=True)}")
            print(f"  Créditos: {locale.currency(suma_valor_cr+suma_valor_cr_tipo4, grouping=True)}")
        
    else:
        print("Validación 3: Los débitos y los créditos no están cuadrados.")

        # Imprimir las sumas finales
        print("\nSuma total de los valores:")
        print(f"  Suma F351_VALOR_DB: {locale.currency(suma_valor_db+suma_valor_db_tipo4, grouping=True)}")
        print(f"  Suma F351_VALOR_CR: {locale.currency(suma_valor_cr+suma_valor_cr_tipo4, grouping=True)}")
        print(f"  Suma F351_VALOR_DB2: {locale.currency(suma_valor_db2+suma_valor_db_tipo4, grouping=True)}")
        print(f"  Suma F351_VALOR_CR2: {locale.currency(suma_valor_cr2+suma_valor_cr_tipo4, grouping=True)}")
        print(f"  Suma F351_VALOR_DB3: {locale.currency(suma_valor_db3+suma_valor_db_tipo4, grouping=True)}")
        print(f"  Suma F351_VALOR_CR3: {locale.currency(suma_valor_cr3+suma_valor_cr_tipo4, grouping=True)}") 

# Función para leer un archivo y validar cada línea
def validar_archivo(archivo):
    global consecutivo_esperado, valores_DB_CR, F350_ID_TIPO_DOCTO_GLOBAL, suma_valor_db_tipo4, suma_valor_cr_tipo4
    registros_tipo1 = []
    registros_tipo2 = []
    registros_tipo3 = []
    registros_tipo4 = []
    registros_tipo5 = []
    registros_tipo6 = []
    
    # Reiniciar variables globales para cada archivo
    F350_ID_TIPO_DOCTO_GLOBAL = None
    suma_valor_db_tipo4 = 0.0
    suma_valor_cr_tipo4 = 0.0
    consecutivo_esperado = 1
    valores_DB_CR = {
        'F351_VALOR_DB': [],
        'F351_VALOR_DB2': [],
        'F351_VALOR_DB3': [],
        'F351_VALOR_CR': [],
        'F351_VALOR_CR2': [],
        'F351_VALOR_CR3': []
    }
    
    errores_archivo = validar_codificacion(archivo)
    if errores_archivo:
        for error in errores_archivo:
            print(error)
        return
    
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for num_linea, linea in enumerate(file, start=1):
            errores = validar_registro(linea.strip().replace("_", " "))  # Sustituir "_" por espacios
            if errores:
                errores_archivo.append(f"Error en la línea {num_linea}: {errores}")
    
        validar_registros_tipo1(registros_tipo1)
        validar_registros_tipo2(registros_tipo2)
        validar_registros_tipo3(registros_tipo3)
        validar_registros_tipo4(registros_tipo4)
        validar_registros_tipo5(registros_tipo5)
        validar_registros_tipo6(registros_tipo6)
    
    if errores_archivo:
        for error in errores_archivo:
            print(error)
        print("El archivo contine errores por solucionar.")
    else:
        print("Validación 2: El archivo cumple con las validaciones.")
    
    # Imprimir los valores al final
    imprimir_valores_finales()

# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    
    # Recorrer todas las subcarpetas y archivos en la carpeta principal
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            # Verificar si el archivo tiene extensión .txt
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)

    return archivos_txt

# Función para procesar todos los archivos .txt y validar su contenido
def procesar_archivos_txt(carpeta):
    archivos = leer_archivos_txt(carpeta)
    
    if not archivos:
        print(f"No se encontraron archivos .txt en la carpeta {carpeta}")
        return
    
    for archivo in archivos:
        print(f"\nProcesando archivo: {archivo}\n")
        validar_archivo(archivo)

procesar_archivos_txt(carpeta)


Procesando archivo: D:\Usuarios\14624165\Desktop\Planos nomina\12 diciembre\20241215\Archivo_Movimiento_Contable_Icesi(20241215,22,21,I,D,0,).txt

Validación 1: Codificación del archivo 'Cumple con ANSI'.
Validación 2: Cumple con fila inicial.
Validación 2: Cumple con fila de encabezado.
Validación 2: Cumple con las filas del movimiento.
Validación 2: Cumple con las filas de cuenta por cobrar.
Validación 2: Cumple con la fila del gasto.
Validación 2: Cumple con la fila final.
Validación 2: El archivo cumple con las validaciones.
Validación 3: Están cuadrados los débitos y los créditos.
  Débitos: $1,750,860,558.00
  Créditos: $1,750,860,558.00

Procesando archivo: D:\Usuarios\14624165\Desktop\Planos nomina\12 diciembre\20241222\Archivo_Movimiento_Contable_Icesi(20241222,22,21,A,0,).txt

Validación 1: Codificación del archivo 'Cumple con ANSI'.
Validación 2: Cumple con fila inicial.
Validación 2: Cumple con fila de encabezado.
Validación 2: Cumple con las filas del movimiento.
Validaci

**ANALISIS DESCRIPTIVO**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

def analisis_descriptivo(df):
    # Estadísticas descriptivas generales
    print("\nEstadísticas descriptivas:")
    print(df[['F351_VALOR_DB', 'F351_VALOR_CR']].describe())
    
    # Distribución de los valores de débito y crédito
    plt.figure(figsize=(10, 6))
    sns.histplot(df['F351_VALOR_DB'], bins=30, color='blue', label='Valor DB', kde=True)
    sns.histplot(df['F351_VALOR_CR'], bins=30, color='red', label='Valor CR', kde=True)
    plt.title("Distribución de Valores de Débito (DB) y Crédito (CR)")
    plt.xlabel("Valor")
    plt.ylabel("Frecuencia")
    plt.legend([],[], frameon=False) 
    plt.show()

    # Comparación de débitos y créditos
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df, x='F351_VALOR_DB', y='F351_VALOR_CR', hue='F350_ID_TIPO_DOCTO', palette='viridis')
    plt.title("Relación entre Valor DB y Valor CR por Tipo de Documento")
    plt.xlabel("Valor DB")
    plt.ylabel("Valor CR")
    plt.legend([],[], frameon=False) 
    plt.show()

    # Gráfico de barras para ver la cantidad de registros por tipo de documento
    plt.figure(figsize=(8, 6))
    sns.countplot(data=df, x='F350_ID_TIPO_DOCTO', palette='muted')
    plt.title("Cantidad de Registros por Tipo de Documento")
    plt.xlabel("Tipo de Documento")
    plt.ylabel("Cantidad de Registros")
    plt.legend([],[], frameon=False) 
    plt.show()

    # Boxplot para ver la distribución de débitos por tipo de documento
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='F350_ID_TIPO_DOCTO', y='F351_VALOR_DB', palette='Set3')
    plt.title("Distribución de Valores de Débito por Tipo de Documento")
    plt.xlabel("Tipo de Documento")
    plt.ylabel("Valor DB")
    plt.legend([],[], frameon=False) 
    plt.show()

    # Boxplot para ver la distribución de créditos por tipo de documento
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='F350_ID_TIPO_DOCTO', y='F351_VALOR_CR', palette='Set2')
    plt.title("Distribución de Valores de Crédito por Tipo de Documento")
    plt.xlabel("Tipo de Documento")
    plt.ylabel("Valor CR")
    plt.legend([],[], frameon=False) 
    plt.show()

# Función principal para consolidar datos y realizar análisis descriptivo
def realizar_analisis(carpeta):
    df = consolidar_archivos(carpeta)
    analisis_descriptivo(df)

# Llamada de prueba para la carpeta con archivos

realizar_analisis(carpeta)

**ENTRENADOR DE ANOMALIAS**


**anomalias_IsolationForest**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el detector de anomalías usando Isolation Forest
def entrenar_detector_anomalias(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el Isolation Forest
    modelo = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    df['anomaly_score'] = modelo.fit_predict(X_scaled)
    
    # Marcar anomalías (donde anomaly_score es -1)
    df['is_anomaly'] = df['anomaly_score'] == -1
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el detector de anomalías
    df_resultados = entrenar_detector_anomalias(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos
#carpeta = (r"D:\Usuarios\14624165\Desktop\Prueba")
resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_IsolationForest.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_IsolationForest.csv'.")

**anomalias_OneClassSVM**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el detector de anomalías usando One-Class SVM
def entrenar_detector_anomalias(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el One-Class SVM
    modelo = OneClassSVM(nu=0.05, kernel='rbf', gamma='scale')
    df['anomaly_score'] = modelo.fit_predict(X_scaled)
    
    # Marcar anomalías (donde anomaly_score es -1)
    df['is_anomaly'] = df['anomaly_score'] == -1
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el detector de anomalías
    df_resultados = entrenar_detector_anomalias(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos
#carpeta = (r"D:\Usuarios\14624165\Desktop\Prueba")
resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_OneClassSVM.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_detectadas.csv'.")

**anomalías_Isolation_Forest**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el detector de anomalías
def entrenar_detector_anomalias(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el Isolation Forest
    modelo = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    df['anomaly_score'] = modelo.fit_predict(X_scaled)
    
    # Marcar anomalías (donde anomaly_score es -1)
    df['is_anomaly'] = df['anomaly_score'] == -1
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Procesar en lotes para evitar MemoryError
    batch_size = 10000  # Ajusta el tamaño del lote según la memoria disponible
    df_resultados = pd.DataFrame()
    
    for start in range(0, len(df), batch_size):
        end = start + batch_size
        df_batch = df.iloc[start:end]
        df_batch_resultados = entrenar_detector_anomalias(df_batch)
        df_resultados = pd.concat([df_resultados, df_batch_resultados], ignore_index=True)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos

resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_detectadas.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_isolation_forest.csv'.")


**anomalias_autoencoder**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para construir el autoencoder
def construir_autoencoder(dimension_entrada):
    modelo = Sequential()
    modelo.add(Dense(16, activation='relu', input_shape=(dimension_entrada,)))
    modelo.add(Dense(8, activation='relu'))
    modelo.add(Dense(4, activation='relu'))
    modelo.add(Dense(8, activation='relu'))
    modelo.add(Dense(16, activation='relu'))
    modelo.add(Dense(dimension_entrada, activation='linear'))  # Capa de salida igual a la entrada
    modelo.compile(optimizer='adam', loss='mse')  # Optimización para minimizar el error cuadrático medio (MSE)
    return modelo

# Función para entrenar el autoencoder y detectar anomalías
def entrenar_autoencoder(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Construir y entrenar el modelo Autoencoder
    modelo = construir_autoencoder(X_scaled.shape[1])
    modelo.fit(X_scaled, X_scaled, epochs=50, batch_size=32, shuffle=True, validation_split=0.1)
    
    # Predecir la reconstrucción de los datos
    predicciones = modelo.predict(X_scaled)
    
    # Calcular el error de reconstrucción
    errores_reconstruccion = np.mean(np.abs(predicciones - X_scaled), axis=1)
    
    # Definir umbral de anomalía (en este caso, 95 percentil de los errores)
    umbral = np.percentile(errores_reconstruccion, 95)
    
    # Marcar anomalías
    df['reconstruction_error'] = errores_reconstruccion
    df['is_anomaly'] = df['reconstruction_error'] > umbral
    
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el autoencoder para la detección de anomalías
    df_resultados = entrenar_autoencoder(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos

resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_autoencoder.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_autoencoder.csv'.")

**anomalias_lof**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el modelo LOF y detectar anomalías
def entrenar_lof_anomalias(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el modelo LOF
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    df['anomaly_score'] = lof.fit_predict(X_scaled)
    
    # Marcar anomalías (donde anomaly_score es -1)
    df['is_anomaly'] = df['anomaly_score'] == -1
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el modelo LOF para la detección de anomalías
    df_resultados = entrenar_lof_anomalias(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos

resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_lof.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_lof.csv'.")

**anomalias_elliptic_envelope**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el modelo Elliptic Envelope y detectar anomalías
from sklearn.covariance import EllipticEnvelope

def entrenar_elliptic_envelope_anomalias(df):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el modelo Elliptic Envelope con un valor más alto de support_fraction
    elliptic_env = EllipticEnvelope(contamination=0.05, support_fraction=0.7, random_state=42)
    df['anomaly_score'] = elliptic_env.fit_predict(X_scaled)
    
    # Marcar anomalías (donde anomaly_score es -1)
    df['is_anomaly'] = df['anomaly_score'] == -1
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el modelo Elliptic Envelope para la detección de anomalías
    df_resultados = entrenar_elliptic_envelope_anomalias(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos

resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_elliptic_envelope.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_elliptic_envelope.csv'.")

**anomalias_kmeans**


In [ ]:
# Función para leer todos los archivos .txt dentro de una carpeta (y subcarpetas)
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame (seleccionando las columnas relevantes)
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()  # Asegúrate de que registro sea una cadena
            # Extraer los campos relevantes
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            
            # Verificar que los valores no estén vacíos antes de convertirlos a float
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,  # Número de línea en el archivo
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Función para consolidar todos los archivos en un único DataFrame
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)  # Agrega el nombre del archivo como referencia
        df['ruta_archivo'] = archivo  # Guarda la ruta completa del archivo
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Función para validar reglas predefinidas
def validar_reglas(df):
    errores = []
    for index, row in df.iterrows():
        # Regla: F351_ID_AUXILIAR debe comenzar con "52" o "50" si F351_ID_CCOSTO comienza con "CA"
        if row['F351_ID_CCOSTO'].startswith("CA") and not row['F351_ID_AUXILIAR'].startswith(("52", "50")):
            errores.append(f"Error en archivo {row['archivo']} en la fila {row['fila']}: F351_ID_AUXILIAR debe comenzar con '52' o '50'")
        # Otras reglas se pueden agregar aquí
    return errores

# Función para entrenar el modelo K-Means y detectar anomalías
def entrenar_kmeans_anomalias(df, n_clusters=5):
    # Selecciona solo las columnas numéricas para el entrenamiento
    columnas_numericas = ['F351_VALOR_DB', 'F351_VALOR_CR']
    X = df[columnas_numericas]
    
    # Normaliza los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar el modelo K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Calcular la distancia de cada punto al centro de su clúster
    df['distancia_centroide'] = np.linalg.norm(X_scaled - kmeans.cluster_centers_[df['cluster']], axis=1)
    
    # Calcular umbral como la media + 2 veces la desviación estándar de las distancias
    umbral = df['distancia_centroide'].mean() + 2 * df['distancia_centroide'].std()
    
    # Marcar anomalías (donde la distancia al centroide es mayor al umbral)
    df['is_anomaly'] = df['distancia_centroide'] > umbral
    return df

# Función principal para detectar anomalías
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar validaciones de reglas
    errores_reglas = validar_reglas(df)
    if errores_reglas:
        print("Errores de reglas detectados:")
        for error in errores_reglas:
            print(error)
    
    # Entrenar y aplicar el modelo K-Means para la detección de anomalías
    df_resultados = entrenar_kmeans_anomalias(df)
    
    # Mostrar resultados de anomalías
    anomalías = df_resultados[df_resultados['is_anomaly'] == True]
    print(f"Se detectaron {len(anomalías)} anomalías.")
    print(anomalías[['archivo', 'fila', 'F351_VALOR_DB', 'F351_VALOR_CR', 'distancia_centroide', 'is_anomaly']])
    
    return df_resultados

# Llamada de prueba para la carpeta con archivos

resultados = detectar_anomalias(carpeta)

# Filtrar las anomalías
anomalías = resultados[resultados['is_anomaly'] == True]

# Guardar las anomalías en un archivo CSV
anomalías.to_csv('anomalias_kmeans.csv', index=False)

print(f"Se guardaron {len(anomalías)} anomalías en el archivo 'anomalias_kmeans.csv'.")

**anomalias_etiquetadas**


In [ ]:
# Función para leer y procesar los archivos
def leer_archivos_txt(carpeta):
    archivos_txt = []
    for ruta_directorio, subdirectorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith(".txt"):
                ruta_completa = os.path.join(ruta_directorio, archivo)
                archivos_txt.append(ruta_completa)
    return archivos_txt

# Función para convertir un archivo en un DataFrame
def procesar_archivo(archivo):
    datos = []
    with open(archivo, 'r', encoding='Windows-1252') as file:
        for i, linea in enumerate(file, start=1):
            registro = linea.strip()
            valor_db = registro[115:136].replace("+", "").strip()
            valor_cr = registro[136:157].replace("+", "").strip()
            if valor_db and valor_cr:
                datos.append({
                    'fila': i,
                    'F351_VALOR_DB': float(valor_db),
                    'F351_VALOR_CR': float(valor_cr),
                    'F350_ID_TIPO_DOCTO': registro[22:25].strip(),
                    'F351_ID_CCOSTO': registro[90:105].strip(),
                    'F351_ID_AUXILIAR': registro[32:52].strip()
                })
    return pd.DataFrame(datos)

# Consolidar archivos
def consolidar_archivos(carpeta):
    archivos = leer_archivos_txt(carpeta)
    df_consolidado = pd.DataFrame()
    for archivo in archivos:
        df = procesar_archivo(archivo)
        df['archivo'] = os.path.basename(archivo)
        df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
    return df_consolidado

# Aplicar LOF para detección de anomalías puntuales
def aplicar_LOF(df):
    lof_model = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    df['lof_score'] = lof_model.fit_predict(df[['F351_VALOR_DB', 'F351_VALOR_CR']])
    df['is_anomaly'] = df['lof_score'] == -1
    return df

# Etiquetar anomalías según el tipo
def etiquetar_anomalias(df):
    df['tipo_anomalia'] = 'Normal'  # Por defecto
    
    for index, row in df.iterrows():
        if row['is_anomaly']:
            # Etiquetar como anomalía puntual
            df.loc[index, 'tipo_anomalia'] = 'Anomalia Puntual'
            
            # Etiquetar anomalías contextuales (ejemplo: si F351_ID_AUXILIAR no empieza con '52' o '50')
            if row['F351_ID_CCOSTO'].startswith('CA') and not row['F351_ID_AUXILIAR'].startswith(('52', '50')):
                df.loc[index, 'tipo_anomalia'] = 'Anomalia Contextual'
    
    return df

# Función principal para aplicar todo el proceso
def detectar_anomalias(carpeta):
    df = consolidar_archivos(carpeta)
    
    # Aplicar LOF para detección de anomalías
    df = aplicar_LOF(df)
    
    # Etiquetar anomalías
    df = etiquetar_anomalias(df)
    
    # Filtrar anomalías
    anomalías = df[df['is_anomaly'] == True]
    
    # Guardar anomalías etiquetadas
    anomalías.to_csv('anomalias_etiquetadas.csv', index=False)
    
    return anomalías

# Llamada de prueba para aplicar detección y etiquetado

resultados = detectar_anomalias(carpeta)
print(resultados)